In [7]:
import numpy as np
from sklearn.metrics import accuracy_score

In [9]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                test_size=0.3, shuffle=True, random_state=42)

In [31]:
from sklearn.metrics import accuracy_score

class RandomForest:
    def __init__(self,B,bootstrap_ratio,max_depth,min_sample_split):
        self.B = B
        self.bootstrap_ratio = bootstrap_ratio
        tree_params = {'max_depth':max_depth,'criterion':'gini','min_samples_split':min_sample_split}
        self.models = [DecisionTreeClassifier(**tree_params) for _ in range(B)]
        self.models_idn = []
        self.accuracy = []
        self.average_accuracy = None

    def fit(self,X_train,y_train):
        sample_size = int(self.bootstrap_ratio*len(X_train))
        m, n = X_train.shape
        nn = int(np.sqrt(n))
        xsamples = np.zeros((self.B, sample_size, nn))
        ysamples = np.zeros((self.B, sample_size))
        oob = []
        #subsamples for each model
        for i in range(self.B):
            idx_range = np.arange(sample_size).astype(float)
            n_range = np.arange(n)
            idn = np.random.choice(n_range,size=nn)
            self.models_idn.append(idn)
            for j in range(sample_size):
                idx = random.randrange(m)
                xsamples[i, j, :] = X_train[idx,idn]
                ysamples[i, j] = y_train[idx]
                idx_range[idx] = -1
            oob.append(idx_range[idx_range != -1])
            
        for i, model in enumerate(self.models):
            _X = xsamples[i, :]
            _y = ysamples[i, :]
            model.fit(_X, _y)
        return self.models
    
    def predict(self,X_test):
        prediction = np.zeros((X_test.shape[0],len(self.models)))
        for i, model in enumerate(self.models):
            idn = self.models_idn[i]
            X_test_model = X_test[:,idn]
            yhat = model.predict(X_test_model)
            prediction[:,i] = yhat
        return stats.mode(prediction.T)[0][0]
#         return self.predictions



B = 100
bootstrap_ratio = 1
max_depth = 4
min_sample_split = 5

rf = RandomForest(B,bootstrap_ratio,max_depth,min_sample_split)

rf.fit(X_train,y_train)

y_hat = rf.predict(X_test)

#print(np.sum(y_hat == y_test)/y_test.size)
print(classification_report(y_test, y_hat))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        13

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

